## 빅데이터 실습

### 다나와 무선청소기 상품분석

#### 데이터 수집

##### 셀레니움 사용
- 무선청소기 검색에 사용할 URL

    https://search.danawa.com/dsearch.php?query=무선청소기&originalQuery=무선청소기&previousKeyword=무선청소기&checkedInfo=N&volumeType=allvs&page=1&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A

    한페이지가 다 로드될때 5초정도의 시간이 필요

In [2]:
## 필수 라이브러리 사용등록
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib import parse # url encode
from tqdm import tqdm
import pandas as pd
import time

In [4]:
## 웹드라이버로 크롬 오픈
driver = webdriver.Chrome()
url = r'https://search.danawa.com/dsearch.php?query=무선청소기&page=1&limit=120&sort=saveDESC&list=list'
driver.get(url)
time.sleep(5.0)

In [30]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [31]:
soup

<html class="js no-touch" lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml"><head>
<title>무선청소기 : 다나와 통합검색</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="무선청소기 통합검색 : 다나와 통합검색" name="title"/>
<meta content="'무선청소기'의 다나와 통합검색 결과입니다." name="description"/>
<meta content="무선청소기, 다나와, 최저가, 가격비교, 검색, 통합검색, 차이슨 무선청소기, 디베아 무선청소기, 삼성 무선청소기, 일렉트로룩스 무선청소기, 샤크 무선청소기, 다이슨 무선청소기, 테팔무선청소기, 샤오미 무선청소기, lg 무선청소기, 유선청소기" name="keywords"/>
<meta content="무선청소기 통합검색 : 다나와 통합검색" property="og:title"/>
<meta content="'무선청소기'의 다나와 통합검색 결과입니다." property="og:description"/>
<meta content="https://img.danawa.com/new/tour/img/logo/sns_danawa.jpg" property="og:image"/>
<meta content="width=1280" name="viewport"/>
<link href="//img.danawa.com/new/danawa_main/v1/img/danawa_favicon.ico" rel="shortcut icon"/>
<link href="//static.danawa.com/css/common.css?v=2402280851" rel="stylesheet" type="text/css"/>
<link hre

In [32]:
prodItems = soup.select('ul.product_list > li.prod_item')

In [33]:
# id="productItem19965230" 식으로 전개되므로 productItem 이 아니면 패스
# 제품명 가져오기, 만약 애드스마트 영역이면 예외발생
prodItems[0].select('p.prod_name > a')[0].text.strip()

'삼성전자 제트 VS20T9257SE'

In [73]:
# 제품스펙 가져오기
prodItems[0].select('div.spec_list')[0].text.strip().replace('\t', '')

'핸디스틱청소기 / 무선 / 흡입+물걸레(동시) / 흡입력: 250W / 소비전력: 620W / 2023년형 / [구성] 거치대: 올인원타워 / 먼지비움 / 충전 / UVC LED / 액서서리수납 / 스탠드거치 / 먼지비움시간: 30초 / 브러쉬: 와이드바닥 / 물걸레: 스팀, 고온, 일반 / 솔형 / 틈새 / 먼지봉투: 2.5L / [배터리] 사용시간: 30분(최대) / 충전시간: 4시간 / 분리형(2개) / 리튬이온 / 25.2V / [청소] 싸이클론흡입 / LED라이트 / 자동물공급 / 스마트인버터모터 / [부가] 스마트폰제어 / 대화형알림창 / 워셔블헤파필터 / 5단계여과 / 색상: 카밍베이지 / 무게: 2.47kg / 크기(가로x세로x깊이): 300x1120x245mm'

In [97]:
# 제품가격 가져오기
# 책에서는 1위 데이터를 가져오도록 했음. 
# 우리는 input 중 두번째 값이 최소값을 담고 있는 것을 확인
prodItems[0].select('input')[1].get('value')

'868000'

In [99]:
# 제품 인트로
prodItems[0].select('p.intro_text')[0].text

'가장 프리미엄 라인업(AX)으로 스팀물걸레 포함 구성, AX9988보다 흡입력만 낮음'

##### 1번 페이지에서 상품 정보 추출

In [34]:
prodData = []
loop = 0

for prodItem in prodItems:
    try:
        prodName = prodItem.select('p.prod_name > a')[0].text.strip()  # 상품명 가져오기
        specList = prodItem.select('div.spec_list')[0].text.strip().replace('\t', '') # 상품 스펙목록 가져오기
        price = prodItem.select('input')[1].get('value') # 최저가 가져오기        
        prodData.append([prodName, specList, price])
        loop += 1
    except:
        pass    

In [27]:
# 결과 확인
prodData

[['세비즈 VD-MOP1',
  '물걸레청소기 / 회전식 / 물걸레전용 / 무선 / 소비전력: 60W / [배터리] 리튬이온 / 2000mAh / 충전시간: 2시간30분 / 사용시간: 40분(최대) / [기능] 물분사 / LED라이트 / 각도조절 / 셀프스탠딩 / [부가] 색상: 화이트 / 무게: 3.2kg / 패드: 극세사(벨크로형)',
  '109000'],
 ['스위스밀리터리 듀얼스핀 SMA-WV850D',
  '물걸레청소기 / 회전식 / 물걸레전용 / 무선 / [배터리] 리튬이온 / 2200mAh / 충전시간: 4시간 / 사용시간: 45분(최대) / [기능] 물분사 / LED라이트 / 각도조절 / 셀프스탠딩 / [부가] 2018년형 / 패드: 극세사(벨크로형) / 크기(가로x세로x깊이): 400x1280x210mm',
  '490000'],
 ['아이리스 IC-FDC1',
  '침구청소기 / 핸디형 / 무선 / 충전시간: 3시간30분 / 사용시간: 20분(최대) / [기능] 청소모드: UV살균, 진동, 흡입 / 진동수: 6000회(분당) / 소비전력: 72W / [부가] 색상: 블랙, 핑크, 화이트 / 무게: 2.1kg / 먼지센서 / 크기(가로x세로x깊이): 254x425.3x153mm',
  '165380'],
 ['나리온 무선 건습식 청소기 30L',
  '업소용청소기 / 건습식 / 무선 / 흡입력: 16000Pa / 소비전력: 200W / [기능] BLDC모터 / 헤파필터 / 먼지통 / 충전시간: 3시간 / 사용시간: 40분(최대) / 이동바퀴 / 자동정지 / [부가] 탱크용량: 30L / 색상: 그린 / 2023년형 / 송풍 / 배터리: 5000mAh, 21V / 배터리잔량표시 / 구성: 건습식노즐, 소파브러쉬, 틈새노즐, 라운드브러쉬',
  '169000'],
 ['보만 코맘스 KB0350',
  '물걸레청소기 / 회전식 / 물걸레전용 / 무선 / [배터리] 리튬이온 / 2400mAh / 충전시간: 3시간 / 사용시간: 1시간(최대) / [

##### 여러페이지 크롤링 하는 방법

- URL
    - https://search.danawa.com/dsearch.php?query=무선청소기&originalQuery=무선청소기&previousKeyword=무선청소기&checkedInfo=N&volumeType=allvs&page=1&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A

In [9]:
## 검색어, 페이지를 변경하면서 URL 생성함수
def getSearchPageUrl(keyword, page):
    ecKeyword = parse.quote(keyword)
    url = f'https://search.danawa.com/dsearch.php?query={ecKeyword}&originalQuery={ecKeyword}&previousKeyword={ecKeyword}&checkedInfo=N&volumeType=allvs&' + \
          f'page={page}&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&' + \
           'recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&' + \
           'quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A'
    return url

In [12]:
## 상품정보 추출하는 함수
def getProdItems(prodItems):
    prodData = []

    for prodItem in prodItems:
        try:
            prodName = prodItem.select('p.prod_name > a')[0].text.strip()  # 상품명 가져오기
            specList = prodItem.select('div.spec_list')[0].text.strip().replace('\t', '') # 상품 스펙목록 가져오기
            price = prodItem.select('input')[1].get('value') # 최저가 가져오기        
            prodData.append([prodName, specList, price])
        except:
            pass
    
    return prodData

In [144]:
# 검색어에 ' '가 있으면 URL에서 링크 클릭이 안되는 문제 발생 --> urlEncode: parse.quote()
getSearchPageUrl('마이크로 sd카드', 5)

'https://search.danawa.com/dsearch.php?query=%EB%A7%88%EC%9D%B4%ED%81%AC%EB%A1%9C%20sd%EC%B9%B4%EB%93%9C&originalQuery=%EB%A7%88%EC%9D%B4%ED%81%AC%EB%A1%9C%20sd%EC%B9%B4%EB%93%9C&previousKeyword=%EB%A7%88%EC%9D%B4%ED%81%AC%EB%A1%9C%20sd%EC%B9%B4%EB%93%9C&checkedInfo=N&volumeType=allvs&page=5&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A'

In [3]:
total_page = 10

for page in tqdm(range(1, total_page+1)):
    # print('페이지 처리') # tqdm 진행바가 여러번 출력
    time.sleep(1)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:10<00:00,  1.00s/it]


In [22]:
## 여러페이지 검색후 크롤링하는 작업
driver = webdriver.Chrome()
# 암묵적으로 웹 자원 로드를 위해 3초정도 대기
driver.implicitly_wait(3.0)

keyword = '무선청소기'
totalPage = 20
prodDataTotal = [] # 최종적으로 저장할 리스트

for page in tqdm(range(11, totalPage +1)):
    # 검색 페이지 이동
    url = getSearchPageUrl(keyword, page)
    driver.get(url)
    # 페이지 로딩이 완료될때까지 5초간 대기
    time.sleep(5)

    # 현재 페이지 HTML 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 상품정보 추출
    prodItems = soup.select('ul.product_list > li.prod_item')
    prodItemList = getProdItems(prodItems) # 리스트로 추출하는 함수

    # 추출된 정보를 prodDataTotal 추가
    prodDataTotal += prodItemList   

100%|██████████| 10/10 [01:57<00:00, 11.79s/it]


In [23]:
dfProdData = pd.DataFrame(prodDataTotal)

In [24]:
dfProdData.columns = ['제품명', '제품스펙', '최저가']

In [25]:
dfProdData

,제품명,제품스펙,최저가
0,삼성전자 제트 VS20T9257SE,핸디스틱청소기 / 무선 / 흡입+물걸레(교체) / 흡입력: 200W / 소비전력: ...,1183600
1,짐머만 ZMC-1902WL,핸디스틱청소기 / 무선 / 흡입형 / 소비전력: 110W / 2020년형 / [구성...,74100
2,포엔코 PC-OP01,욕실청소기 / 무선 / 전동회전 / [배터리] 충전시간: 1시간 / 리튬이온 / 1...,43530
3,삼성전자 제트 VS20T9222K2,핸디스틱청소기 / 무선 / 흡입+물걸레(교체) / 흡입력: 200W / 소비전력: ...,1323400
4,앱코 오엘라 네오스핀 OBC-AW09 + OST-AWUVCGY,욕실청소기 / 무선 / 전동회전 / 회전속도: 220RPM / 소비전력: 8W / ...,84900
...,...,...,...
914,미니센 무선 소형 청소기 MNC-100 먼지통,차량용청소기 / 먼지통,7000
915,프리라벨 2in1 무선 진공 청소기 LS-2000 헤파필터,차량용청소기 / 필터 / [필터] 헤파필터,3500
916,상품상세설명 참조 캐치웰 텀블러 디자인 초경량 무선 핸디형 진공청소기_아이언그레이,,//상품상세설명 참조 캐치웰 텀블러 디자인 초경량 <b>무선</b> 핸디형 진공<b...
917,포쉬 워시젯 V2 WJ200,침구청소기 / 핸디형 / 무선 / 사용시간: 25분(최대) / [기능] 청소모드: ...,219000


In [28]:
dfProdData.to_excel('./data/다나와_무선청소기_결과2.xlsx', index=False)